In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from joblib import dump

In [2]:
df_data = pd.read_csv('EGFR_Feature_Extraction.csv')
df_data.head()

,molecule_chembl_id,canonical_smiles,Molecular Weight,Number of Rotatable Bonds,Number of Atoms,Number of Bonds,Count of Chiral Centers,Number of Rings,Number of Aromatic Rings,Number of Hydrogen Bond Donors,Number of Hydrogen Bond Acceptors,Balaban J Index,Wiener Index,LogP,TPSA,standard_value,class
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,383.094916,3,27,30,0,4,3,3,4,1.789174,1832.0,4.45034,82.70,41.0,active
1,CHEMBL137635,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,369.170194,6,28,31,0,4,4,1,6,1.558763,2326.0,4.77200,78.66,9300.0,inactive
2,CHEMBL306988,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,283.095691,5,21,21,0,1,1,2,4,2.782020,1108.0,2.31056,113.98,500000.0,inactive
3,CHEMBL66879,O=C(O)/C=C/c1ccc(O)cc1,164.047344,2,12,12,0,1,1,2,2,2.772026,223.0,1.49000,57.53,3000000.0,inactive
4,CHEMBL77085,N#CC(C#N)=Cc1cc(O)ccc1[N+](=O)[O-],215.033091,2,16,16,0,1,1,1,5,3.386653,441.0,1.73096,110.95,96000.0,inactive


In [3]:
df = df_data.drop(df_data.columns[-2], axis=1, inplace=True)
df = df_data.iloc[:, 2:]
print("Null values: " + str(df.isnull().values.any()))
print(df['class'].value_counts())
df.head()

Null values: False
class
active      3810
inactive    3524
Name: count, dtype: int64


,Molecular Weight,Number of Rotatable Bonds,Number of Atoms,Number of Bonds,Count of Chiral Centers,Number of Rings,Number of Aromatic Rings,Number of Hydrogen Bond Donors,Number of Hydrogen Bond Acceptors,Balaban J Index,Wiener Index,LogP,TPSA,class
0,383.094916,3,27,30,0,4,3,3,4,1.789174,1832.0,4.45034,82.70,active
1,369.170194,6,28,31,0,4,4,1,6,1.558763,2326.0,4.77200,78.66,inactive
2,283.095691,5,21,21,0,1,1,2,4,2.782020,1108.0,2.31056,113.98,inactive
3,164.047344,2,12,12,0,1,1,2,2,2.772026,223.0,1.49000,57.53,inactive
4,215.033091,2,16,16,0,1,1,1,5,3.386653,441.0,1.73096,110.95,inactive


In [4]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [5]:
le = LabelEncoder()
y = le.fit_transform(y)

In [6]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, random_state=42)

In [9]:
model = XGBClassifier(objective='binary:logistic', tree_method='gpu_hist')

eval_set = [(X_train, y_train), (X_val, y_val)]

model.fit(
    X_train,
    y_train,
    eval_set=eval_set,
    verbose=True
)

/home/jjk339/miniconda3/envs/FirstProject/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [11:25:26] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[0]	validation_0-logloss:0.61958	validation_1-logloss:0.62400
[1]	validation_0-logloss:0.57331	validation_1-logloss:0.58858
[2]	validation_0-logloss:0.53910	validation_1-logloss:0.55827
[3]	validation_0-logloss:0.50886	validation_1-logloss:0.52945
[4]	validation_0-logloss:0.48780	validation_1-logloss:0.51213
[5]	validation_0-logloss:0.47331	validation_1-logloss:0.50084
[6]	validation_0-logloss:0.46258	validation_1-logloss:0.49512
[7]	validation_0-logloss:0.45309	validation_1-logloss:0.48966
[8]	validation_0-logloss:0.44132	validation_1-logloss:0.48503
[9]	validation_0-logloss:0.42563	validation_1-logloss:0.47682
[10]	validation_0-logloss:0.41182	validation_1-logloss:0.46910
[11]	validation_0-logloss:0.40458	validation_1-logloss:0.46705
[12]	validation_0-logloss:0.39678	validation_1-logloss:0.46418
[13]	validation_0-logloss:0.38987	validation_1-logloss:0.46237
[14]	validation_0-logloss:0.38593	validation_1-logloss:0.45970
[15]	validation_0-logloss:0.37604	validation_1-logloss:0.45589
[1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [10]:
y_pred = model.predict(X_test)
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.80      0.84      0.82       764
           1       0.82      0.78      0.80       703

    accuracy                           0.81      1467
   macro avg       0.81      0.81      0.81      1467
weighted avg       0.81      0.81      0.81      1467



/home/jjk339/miniconda3/envs/FirstProject/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [11:25:58] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/jjk339/miniconda3/envs/FirstProject/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [11:25:58] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [12]:
saved_model = "MainXGBClassifierEGFR.joblib"
dump(model, saved_model)

['MainXGBClassifierEGFR.joblib']